In [ ]:
import pandas as pd
import numpy as np
from scanpy import AnnData
import scanpy as sc
import cobra as cb
import numpy as np
import time
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from utils import find_bh

In [ ]:
npcs=[5,10,15,20]
neigh=[5,10,15,20]
letters=["A","B","C"]
titles=["GSE110949","E-GEOD-86618","GSE118056"]

In [ ]:
fig,axes=plt.subplots(3,3,figsize= (20,14))
axes=axes.ravel()

In [ ]:

j=0
for file_input in titles:
    for apply_denoising in [True,False]:
    
        adata=sc.read_h5ad("outputs/readcounts/dataset"+file_input+"_nodenoise")

        if apply_denoising:
            sc.external.pp.magic(adata,random_state=0)   

        sc.pp.log1p(adata)
        sc.pp.scale(adata)

        if file_input=="E-GEOD-86618":
            cell_cycle_genes = [x.strip() for x in open('data/cellcycle_genes_ensg.txt')]
        else:
            cell_cycle_genes = [x.strip() for x in open('data/cellcycle_genes_genesymbol.txt')]
        s_genes = cell_cycle_genes[:43]
        g2m_genes = cell_cycle_genes[43:]
        cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

        s_genes = [x for x in s_genes if x in adata.var_names]
        g2m_genes = [x for x in g2m_genes if x in adata.var_names]


        sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

        sc.tl.pca(adata, svd_solver='arpack')
        df=find_bh(adata,n_pcs=npcs,n_neighbors=neigh)
        el=df["cluster_values_sil"].argmax()
        df.iloc[el,:]
        n_pcs=int(df.iloc[el]["pcs_values"])
        neigh_values=int(df.iloc[el]["neigh_values"])
        
        sc.pp.neighbors(adata, n_neighbors=neigh_values ,n_pcs=n_pcs)
        sc.tl.umap(adata,random_state=0)

        if file_input=="E-GEOD-86618":
            values=adata.obs['Factor Value[disease]'].values
            values=[el if el=="normal" else "IFP" for el in values ]
            adata.obs['Factor Value[disease]']=values
            
            if apply_denoising:
                sc.pl.umap(adata, color=['Factor Value[disease]'],title="",return_fig=False,show=False,ax=axes[j+6])  
            else:
                sc.pl.umap(adata, color=['Factor Value[disease]'],title="",return_fig=False,show=False,ax=axes[j])  
        else:
            if apply_denoising:
                sc.pl.umap(adata, color=['Type',],title="",return_fig=False,show=False,ax=axes[j+6])
            else:
                sc.pl.umap(adata, color=['Type',],title="",return_fig=False,show=False,ax=axes[j])
                
        if not apply_denoising:
            sc.pl.umap(adata, color=['phase'],title="",return_fig=False,show=False,ax=axes[j+3])

    j=j+1      

In [ ]:
axes[0].annotate(letters[0], xy=(-0.1, 0.9), xycoords="axes fraction",fontsize=30)
axes[3].annotate(letters[1], xy=(-0.1, 0.9), xycoords="axes fraction",fontsize=30)
axes[6].annotate(letters[2], xy=(-0.1, 0.9), xycoords="axes fraction",fontsize=30)
axes[0].set_title(titles[0],fontsize=20)
axes[1].set_title(titles[1],fontsize=20)
axes[2].set_title(titles[2],fontsize=20)

In [ ]:
#axes.reshape(-3,3)
fig.tight_layout()
plt.show()

In [ ]:
fig

In [ ]:
fig.savefig("images/readcounts.png",dpi=300)